In [16]:
from dataloader import get_dataloader


ImportError: cannot import name 'get_dataloader' from 'dataloader' (/home/vincent/unifr/master_thesis/master_lit/dataloader.py)

## DataSet

In [13]:
batch_size = 2
shuffle_img = False
num_workers = 2
split = "train"

In [14]:

dataloader_train = get_dataloader(batch_size=batch_size,shuffle=shuffle_img,num_workers=num_workers,split=split)

NameError: name 'get_dataloader' is not defined